# BÜYÜK VERİDE MAKİNE ÖĞRENMESİ
### Uçtan uca makine öğrenmesi örneği

In [115]:
import findspark

In [116]:
findspark.init("C:\spark")

In [117]:
from pyspark import SparkContext

In [118]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [119]:
# Spark bağlantısının yapılandırılması

spark = SparkSession.builder \
    .master("local") \
        .appName("pyspark_giris") \
            .getOrCreate()

# Bağlantının oluşturulması
sc = spark.sparkContext

In [120]:
sc

<SparkContext master=local appName=pyspark_giris>

In [121]:
spark_df = spark.read.csv("./verisetleri/churn.csv", header=True, inferSchema=True)

In [122]:
spark_df.show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [11]:
# Değişken isimleri hepsi küçük yada hepsi büyük olacak şekilde değiştirilebilir. Yada camel tarzı kullanılabilir.

In [123]:
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

In [124]:
spark_df.show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           names| age|total_purchase|account_manager|years|num_sites|churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [125]:
spark_df = spark_df.withColumnRenamed("_C0", "index")

In [126]:
spark_df

DataFrame[index: int, names: string, age: double, total_purchase: double, account_manager: int, years: double, num_sites: double, churn: int]

In [127]:
spark_df.count()

900

In [128]:
len(spark_df.columns)

8

In [129]:
spark_df.columns

['index',
 'names',
 'age',
 'total_purchase',
 'account_manager',
 'years',
 'num_sites',
 'churn']

In [16]:
spark_df.describe().show()

+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|             index|        names|              age|   total_purchase|   account_manager|            years|         num_sites|              churn|
+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|               900|          900|              900|              900|               900|              900|               900|                900|
|   mean|             449.5|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|259.95191863111916|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|                 0|   Aaron King|             22.0|          

In [130]:
spark_df.select("age", "total_purchase", "account_manager", "years", "num_sites", "churn").describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,900,41.81666666666667,6.127560416916251,22.0,65.0
total_purchase,900,10062.82403333334,2408.644531858096,100.0,18026.01
account_manager,900,0.4811111111111111,0.4999208935073339,0,1
years,900,5.27315555555555,1.274449013194616,1.0,9.15
num_sites,900,8.587777777777777,1.7648355920350969,3.0,14.0
churn,900,0.16666666666666666,0.3728852122772358,0,1


In [23]:
# Sınıflandırma problemi. Müşteriler bizi terk edecek mi etmeyecek mi

In [25]:
# VERİ ÖNİŞLEME

In [131]:
spark_df = spark_df.dropna()

In [27]:
# Varolan değişken üzerinden değişken türetip yeni değişken ekleme

In [132]:
spark_df = spark_df.withColumn("age_kare", spark_df.age**2)

In [133]:
spark_df.show(3)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
only showing top 3 rows



In [30]:
# Büyük veri dünyasında özellikle makine öğrenmesi tarafında algoritmaları kullanırken bağımlı değişkeni ve bağımsız değişkenleri özel bir şekilde ifade etmek gerekmektedir.
# Burada bağımlı değişkenimizi sanki integer mış gibi düşünüp - ki siz bunu başka problemlere uyguladığınızda elinizde string formunda olan bağımlı değişkenler de olabilecektir. Sanki string miş gibi düşünüp bir string bağımlı değişkeni oluşturma işlemi gerçekleştirmiş olacağız. 

## Bağımli değişkeni belirtme, dönüştürme yani ayarlanması

In [134]:
from pyspark.ml.feature import StringIndexer

In [135]:
# Bağımlı değişken belirlenmekte ve "label" olarak etiketlenmektedir.
stringIndexer = StringIndexer(inputCol = "churn", outputCol = "label")

In [136]:
# Yukarıdaki değişikliği spark_df e uygulanıyor.
mod = stringIndexer.fit(spark_df)

# İndexlerin integer a göre dönüştürülmesi
indexed = mod.transform(spark_df)
spark_df = indexed.withColumn("label", indexed["label"].cast("integer"))

In [137]:
spark_df.show(5)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|  1764.0|    1|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|  1369.0|    1|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
only showing top 5 rows



## Bağımsız değişkenlerin ayarlanması

In [138]:
from pyspark.ml.feature import VectorAssembler

In [26]:
spark_df.columns

['index',
 'names',
 'age',
 'total_purchase',
 'account_manager',
 'years',
 'num_sites',
 'churn',
 'age_kare',
 'label']

In [139]:
bagimsiz_degiskenler = ["age", "total_purchase", "account_manager", "years", "num_sites"]

In [28]:
# Tüm değişkenleri sanki tek bir vektördeymişçesine dönüştürme işlemine tabi tutmamız gerekiyor. Yani bir gözlemin değişken değerleri tek boyutlu bir matrise dönüştürülüyor.
vectorAssembler = VectorAssembler(inputCols = bagimsiz_degiskenler, outputCol = "features")

In [140]:
va_df = vectorAssembler.transform(spark_df)

In [30]:
va_df.show(5)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|            features|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|[42.0,11066.8,0.0...|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|[41.0,11916.22,0....|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|[38.0,12884.75,0....|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|  1764.0|    1|[42.0,8010.76,0.0...|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|  1369.0|    1|[37.0,9191.58,0.0...|
+-----+----------------+----+--------------+----

### SONUÇ VERİ SETİNİN OLUŞTURULMASI LABEL VE FEATURES IN ÇIKARILMASI

In [141]:
final_df = va_df.select(["features","label"])

In [142]:
final_df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
+--------------------+-----+
only showing top 5 rows



### TEST - TRAIN AYRIMININ YAPILMASI

In [143]:
splits = final_df.randomSplit([0.70, 0.30])

In [144]:
train_df = splits[0]

In [145]:
test_df = splits[1]

In [146]:
train_df

DataFrame[features: vector, label: int]

In [147]:
test_df

DataFrame[features: vector, label: int]

In [53]:
# BÖYLECE VERİ MODELLEMEYE HAZIR HALE GETİRİLDİ.

## GBM ( Gradient Boosting Machine ) ile Müşteri Terk Modellemesi

In [148]:
from pyspark.ml.classification import GBTClassifier

In [149]:
gbm = GBTClassifier(maxIter = 10, featuresCol = "features", labelCol = "label")

In [150]:
gbm_model = gbm.fit(train_df)

In [151]:
sc

<SparkContext master=local appName=pyspark_giris>

In [42]:
# Arayüze bakıldığında arka planda RandomForest ve DecisionTree gibi yapıların açıştırıldığını görebilirsiniz.

In [43]:
# TAHMİN ETME

In [44]:
y_pred = gbm_model.transform(test_df)

In [45]:
# İlkel test hatasının hesaplanması

In [48]:
y_pred

DataFrame[features: vector, label: int, rawPrediction: vector, probability: vector, prediction: double]

In [46]:
# y_pred içerisinden label lar ve tahmin edilen değerlerin seçilmesi
ac = y_pred.select("label", "prediction")

In [51]:
# Gerçek değerler ile (ac.label), tahmin edilen değerleri karşılaştıracak (ac.prediction), True dönen değerleri yani doğru tahmin edilen değerlerin sayısını toplam sayıya bölecek. Böylece accuracy değeri hesaplanacak
ac.filter(ac.label == ac.prediction).count() / ac.count()

0.8514492753623188

 ## MODEL TUNNING

In [53]:
# MODEL OPTİMİZAZYONU İÇİN GİRMEMİZ GEREKEN, MODEL NESNESİ VE PARAMETRE VB. AYARLAR

# Binary classification değerlendirmesine yönelik fonksiyon
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# PaamGridBuilder : Parametre gridleri için fonksiyon
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Başarı değerlendirmesi için sınıflandırma nesnesi. İkili sınıflandırma (yani 0,1; yani evet, hayır) yaptığımız için bunu kullanıyoruz.
evaluator = BinaryClassificationEvaluator()

paramGrid = (ParamGridBuilder()
.addGrid(gbm.maxDepth, [2, 4,6])
.addGrid(gbm.maxBins, [20, 30])
.addGrid(gbm.maxIter, [10, 20])
.build())

# estimator--> tahminci gbm nesnesi
# evaluator--> başarıyı ölçmek için kullanıcak yol-nesne-fonksiyon
# numFolds --> Kaç katlı cross validation yapılacak
cv = CrossValidator(estimator = gbm, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = 10)


In [54]:
# FİT ETME : İşlem devam ederken arayüzden çalışan işlemleri takip edebilirsiniz.
cvModel = cv.fit(train_df)

In [55]:
# TAHMİN ETME VE HATA BULMA

In [56]:
y_pred = cvModel.transform(test_df)

In [57]:
ac = y_pred.select("label", "prediction")

In [58]:
ac.filter(ac.label == ac.prediction).count()/ac.count()

0.8804347826086957

In [59]:
# Yaklaşık olarak % 90 başarı ile tahmainler yapılabilmektedir

In [60]:
# Yeni Müşteri Terk Eder mi? Etmez Mİ?

## MODELİN KULLANILMASI : YENİ VERİ ÜZERİNDE TEST EDİLMESİ

In [103]:
import pandas as pd

names = pd.Series(["Celal AKSU", "Taner Gün", "Eylem", "Berkay", "Özge"])
age = pd.Series([38, 43, 34, 50, 40])
total_purchase = pd.Series([30000, 10000, 6000, 30000, 100000])
account_manager = pd.Series([1, 0, 0, 1, 1])
years = pd.Series([20, 10, 3, 8, 30])
num_sites = pd.Series([30, 8, 8, 6, 50,])

yeni_musteriler = pd.DataFrame({'names':names,
'age':age,
'total_purchase':total_purchase,
'account_manager':account_manager,
'years':years,
'num_sites':num_sites})

yeni_musteriler.columns

Index(['names', 'age', 'total_purchase', 'account_manager', 'years',
       'num_sites'],
      dtype='object')

In [63]:
# Bu verileri büyük veri ortamının anlayacağı forma sokmamız ve büyük veri ortamına göndermemiz gerekir.

In [64]:
# Biz burada kendi localimizde el yordamıyla bir pandas dataframe oluşturduk. Bunu kurmuş olduğumuz spark makine öğrenmesi model nesnesiyle tahmin edebilmek adına bunu spark ın analayabileceği bir dile çevirmemiz gerekiyor. Yani SPARK DATAFRAME ini çevirmemiz gerekiyor.

In [104]:
yeni_sdf = spark.createDataFrame(yeni_musteriler)

In [105]:
yeni_sdf.show()

+----------+---+--------------+---------------+-----+---------+
|     names|age|total_purchase|account_manager|years|num_sites|
+----------+---+--------------+---------------+-----+---------+
|Celal AKSU| 38|         30000|              1|   20|       30|
| Taner Gün| 43|         10000|              0|   10|        8|
|     Eylem| 34|          6000|              0|    3|        8|
|    Berkay| 50|         30000|              1|    8|        6|
|      Özge| 40|        100000|              1|   30|       50|
+----------+---+--------------+---------------+-----+---------+



In [68]:
# Spark dataframe e çevirme işlemi başarılı olduğu gözlenmektedir.

In [69]:
# Bu verileri ayrıca model nesnesinin anlayacağı formata çevirmemiz gerekiyor.

In [70]:
# Yani bağımsız değikenleri tek bir vektöre çevirmemiz gerekir.

In [106]:
yeni_musteriler = vectorAssembler.transform(yeni_sdf)

In [107]:
yeni_musteriler.show()

+----------+---+--------------+---------------+-----+---------+--------------------+
|     names|age|total_purchase|account_manager|years|num_sites|            features|
+----------+---+--------------+---------------+-----+---------+--------------------+
|Celal AKSU| 38|         30000|              1|   20|       30|[38.0,30000.0,1.0...|
| Taner Gün| 43|         10000|              0|   10|        8|[43.0,10000.0,0.0...|
|     Eylem| 34|          6000|              0|    3|        8|[34.0,6000.0,0.0,...|
|    Berkay| 50|         30000|              1|    8|        6|[50.0,30000.0,1.0...|
|      Özge| 40|        100000|              1|   30|       50|[40.0,100000.0,1....|
+----------+---+--------------+---------------+-----+---------+--------------------+



In [108]:
results = cvModel.transform(yeni_musteriler)

In [110]:
results.select("names", "prediction").show()

+----------+----------+
|     names|prediction|
+----------+----------+
|Celal AKSU|       1.0|
| Taner Gün|       0.0|
|     Eylem|       0.0|
|    Berkay|       0.0|
|      Özge|       1.0|
+----------+----------+



In [111]:
# 1 terkedecek, 0 terketmeyecek

In [112]:
sc

<SparkContext master=local appName=pyspark_giris>

In [152]:
sc.stop()